**Recommendation 알고리즘**  
popularity에 따른 item을 추천 가능

In [1]:
ratings = {
    'Dave': {'달콤한 인생':5, '범죄도시':3, '샤인':3},
    'David': {'달콤한 인생':5, '범죄도시':1, '샤인':4},
    'Alex': {'달콤한 인생 ':0, '범죄도시':4, '샤인':5},
    'Andy': {'달콤한 인생': 2, '범죄도시':1, '샤인':5}
}

평점이 높은 두 영화 출력

In [2]:
movie_dict = dict()
for rating in ratings:
    for movie in ratings[rating].keys():
        if movie not in movie_dict:
            movie_dict[movie] = ratings[rating][movie]
        else:
            movie_dict[movie] = (movie_dict[movie] + ratings[rating][movie]) 

for movie in ratings[rating].keys():
    movie_dict[movie] = movie_dict[movie] / 4

import operator
sorted_x = sorted(movie_dict.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_x[:2])

[('샤인', 4.25), ('달콤한 인생', 3.0)]


**Collaborative Filtering**  
1.데이터 구성 / 사용자-항목 선호도(item) 행렬을 만든다.  
2.유사도계산  
3.추천목록 생성: Top N 개의 추천목록 생성

In [3]:
ratings={
     'Dave':{'달콤한인생':5,'범죄도시':3,'샤인':3},
     'David':{'달콤한인생':2,'범죄도시':1,'샤인':4},
     'Alex':{'범죄도시':4,'샤인':5},
     'Andy':{'달콤한인생':2,'범죄도시':1,'샤인':5}
}

In [4]:
ratings.get('Dave')

{'달콤한인생': 5, '범죄도시': 3, '샤인': 3}

In [5]:
ratings.get('Dave').get('샤인')

3

유사도 계산(피타고라스 공식을 통해 거리 계산)

In [6]:
import math
def sim(i,j):
    return math.sqrt(pow(i,2)+pow(j,2))

In [7]:
var1= ratings['Alex']['범죄도시'] - ratings['Andy']['범죄도시']

In [8]:
var1

3

In [9]:
var2 =  ratings['Alex']['샤인'] - ratings['Andy']['샤인']

In [10]:
var2

0

In [11]:
sim(var1, var2)

3.0

Alex가 평가한 범죄도시, 샤인을 모두 평가한 사용자와 모두 거리 구하기(다른 사용자들간의 유사도 구하기)

In [13]:
for i in ratings:
    if i!='Alex':
        num1 = ratings.get('Alex').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Alex').get('샤인') - ratings.get(i).get('샤인')
        print(i,":", sim(num1,num2))

Dave : 2.23606797749979
David : 3.1622776601683795
Andy : 3.0


정규화를 통해 유사도 범위를 0과 1사이에 가두고, 1에 가까울수록 유사도가 높게 변경

In [15]:
for i in ratings:
    if i!='Alex':
        num1 = ratings.get('Alex').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Alex').get('샤인') - ratings.get(i).get('샤인')
        print(i,":", 1 / ( 1 + sim(num1,num2) ) )

Dave : 0.3090169943749474
David : 0.2402530733520421
Andy : 0.25


평균제곱차이 유사도

In [16]:
def sim_msd(data, name1, name2):
    sum = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum += pow(data[name1][movies]- data[name2][movies], 2)
            count += 1

    return 1 / ( 1 + (sum / count) )

In [17]:
sim_msd(ratings, 'Dave','Alex')

0.2857142857142857

In [18]:
sim_msd(ratings, 'Dave','Andy') #Dave와 Andy의 msd 유사도 구하기

0.15

코사인 유사도

In [19]:
import math
def sim_cosine(data, name1, name2):
    sum_name1 = 0
    sum_name2 = 0
    sum_name1_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies], 2)
            sum_name2 += pow(data[name2][movies], 2)
            sum_name1_name2 += data[name1][movies]*data[name2][movies]
    
    return sum_name1_name2 / (math.sqrt(sum_name1)*math.sqrt(sum_name2))

In [20]:
sim_cosine(ratings, 'Dave','Alex')

0.9938837346736189

In [21]:
sim_cosine(ratings, 'Dave','David')

0.8319479070496523

In [22]:
sim_cosine(ratings, 'Dave','Andy')

0.7795844649455863

피어슨 유사도

In [23]:
def sim_pearson(data, name1, name2):
    avg_name1 = 0
    avg_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            avg_name1 = data[name1][movies]
            avg_name2 = data[name2][movies]
            count += 1
    
    avg_name1 = avg_name1 / count
    avg_name2 = avg_name2 / count
    
    sum_name1 = 0
    sum_name2 = 0
    sum_name1_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies] - avg_name1, 2)
            sum_name2 += pow(data[name2][movies] - avg_name2, 2)
            sum_name1_name2 += (data[name1][movies] - avg_name1) * (data[name2][movies] - avg_name2)
    
    return sum_name1_name2 / (math.sqrt(sum_name1)*math.sqrt(sum_name2))

In [24]:
sim_pearson(ratings, 'Dave','Alex')

0.970142500145332

In [25]:
def top_match(data, name, index=3, sim_function=sim_pearson):
    li=[]
    for i in data: #딕셔너리를 돌고
        if name!=i: #자기 자신이 아닐때만
            li.append((sim_function(data,name,i),i)) #sim_function()을 통해 상관계수를 구하고 li[]에 추가
    li.sort() #오름차순
    li.reverse() #내림차순
    return li[:index]

In [26]:
top_match(ratings, 'Dave', 3)

[(0.970142500145332, 'Alex'),
 (0.5406205059012895, 'David'),
 (0.39840953644479793, 'Andy')]

In [27]:
top_match(ratings, 'Dave', 3, sim_function=sim_msd)

[(0.2857142857142857, 'Alex'), (0.1764705882352941, 'David'), (0.15, 'Andy')]

In [28]:
top_match(ratings, 'Dave', 3, sim_function=sim_cosine)

[(0.9938837346736189, 'Alex'),
 (0.8319479070496523, 'David'),
 (0.7795844649455863, 'Andy')]

In [29]:
top_match(ratings, 'Dave', 3, sim_function=sim_pearson)

[(0.970142500145332, 'Alex'),
 (0.5406205059012895, 'David'),
 (0.39840953644479793, 'Andy')]

<h3>유사도와 KNN을 활용한 예측 값 계산 및 추천 목록 생성 기법</h3>
:사용자들 간의 유사도를 바탕으로 모든 항목에 대해 예측 값을 계산하고 높은 예측 값을 갖는 상위 N개의 추천 목록 생성

K Nearest Neighbors(KNN) 가중치 예측 기법

-유사도가 구해지면 평점을 예측하고자 하는 사용자(또는 상품)과 유사도가 큰 k개의 사용자(또는 상품) 벡터를 사용하여 가중 평균을 구해서 가중치를 예측

1. 대상과 가장 유사도가 높은 k의 대상의 영화 평점과 유사도를 통해 유측평점((유사도x(타인의)영화평점)을 구한다.
2. 추측평점의 총합을 구함
3. 추측평점 총합계/유사도 합계를 통해 예상평점을 뽑아낼 수 있다.

In [30]:
ratings_expand = {
    '마동석': {
        '택시운전사': 3.5,
        '남한산성': 1.5,
        '킹스맨:골든서클': 3.0,
        '범죄도시': 3.5,
        '아이 캔 스피크': 2.5,
        '꾼': 3.0,
    },
    '이정재': {
        '택시운전사': 5.0,
        '남한산성': 4.5,
        '킹스맨:골든서클': 0.5,
        '범죄도시': 1.5,
        '아이 캔 스피크': 4.5,
        '꾼': 5.0,
    },
    '윤계상': {
        '택시운전사': 3.0,
        '남한산성': 2.5,
        '킹스맨:골든서클': 1.5,
        '범죄도시': 3.0,
        '꾼': 3.0,
        '아이 캔 스피크': 3.5,
    },
    '설경구': {
        '택시운전사': 2.5,
        '남한산성': 3.0,
        '범죄도시': 4.5,
        '꾼': 4.0,
    },
    '최홍만': {
        '남한산성': 4.5,
        '킹스맨:골든서클': 3.0,
        '꾼': 4.5,
        '범죄도시': 3.0,
        '아이 캔 스피크': 2.5,
    },
    '홍수환': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '킹스맨:골든서클': 1.0,
        '범죄도시': 3.0,
        '꾼': 3.5,
        '아이 캔 스피크': 2.0,
    },
    '나원탁': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '꾼': 3.0,
        '범죄도시': 5.0,
        '아이 캔 스피크': 3.5,
    },
    '소이현': {
        '남한산성': 4.5, 
        '아이 캔 스피크': 1.0,
        '범죄도시': 4.0
    }
}

In [31]:
def getRecommendation (data, person, k=3, sim_function=sim_pearson):
    
    result = top_match(data, person, k)
    
    score = 0 # 평점 합을 위한 변수
    li = list() # 리턴을 위한 리스트
    score_dic = dict() # 유사도 총합을 위한 dic
    sim_dic = dict() # 평점 총합을 위한 dic

    for sim, name in result: # 튜플이므로 한번에
        print(sim, name)
        if sim < 0 : continue #유사도가 양수인 사람만
        for movie in data[name]: 
            if movie not in data[person]: #name이 평가를 내리지 않은 영화
                score += sim * data[name][movie] # 그사람의 영화평점 * 유사도
                score_dic.setdefault(movie, 0) # 기본값 설정
                score_dic[movie] += score # 합계 구함

                # 조건에 맞는 사람의 유사도의 누적합을 구한다
                sim_dic.setdefault(movie, 0) 
                sim_dic[movie] += sim

            score = 0  #영화가 바뀌었으니 초기화한다
    
    for key in score_dic: 
        score_dic[key] = score_dic[key] / sim_dic[key] # 평점 총합/ 유사도 총합
        li.append((score_dic[key],key)) # list((tuple))의 리턴을 위해서.
    li.sort() #오름차순
    li.reverse() #내림차순
    return li

In [32]:
getRecommendation(ratings_expand, '소이현')

0.9330597055272909 홍수환
0.8909876971472571 최홍만
0.8452277090445156 나원탁


[(3.675468553454334, '꾼'),
 (3.0000000000000004, '택시운전사'),
 (1.976934805357391, '킹스맨:골든서클')]

In [33]:
getRecommendation(ratings_expand, '최홍만', k=2, sim_function=sim_cosine)

0.9608329054174726 홍수환
0.9517663735117331 소이현


[(3.0, '택시운전사')]

In [34]:
for name in ratings_expand:
    sum = 0
    count = 0
    for movies in ratings_expand[name]:
        sum += ratings_expand[name][movies]
        count += 1
    ratings_expand[name]['avg'] = sum / count

In [35]:
ratings_expand

{'마동석': {'택시운전사': 3.5,
  '남한산성': 1.5,
  '킹스맨:골든서클': 3.0,
  '범죄도시': 3.5,
  '아이 캔 스피크': 2.5,
  '꾼': 3.0,
  'avg': 2.8333333333333335},
 '이정재': {'택시운전사': 5.0,
  '남한산성': 4.5,
  '킹스맨:골든서클': 0.5,
  '범죄도시': 1.5,
  '아이 캔 스피크': 4.5,
  '꾼': 5.0,
  'avg': 3.5},
 '윤계상': {'택시운전사': 3.0,
  '남한산성': 2.5,
  '킹스맨:골든서클': 1.5,
  '범죄도시': 3.0,
  '꾼': 3.0,
  '아이 캔 스피크': 3.5,
  'avg': 2.75},
 '설경구': {'택시운전사': 2.5, '남한산성': 3.0, '범죄도시': 4.5, '꾼': 4.0, 'avg': 3.5},
 '최홍만': {'남한산성': 4.5,
  '킹스맨:골든서클': 3.0,
  '꾼': 4.5,
  '범죄도시': 3.0,
  '아이 캔 스피크': 2.5,
  'avg': 3.5},
 '홍수환': {'택시운전사': 3.0,
  '남한산성': 4.0,
  '킹스맨:골든서클': 1.0,
  '범죄도시': 3.0,
  '꾼': 3.5,
  '아이 캔 스피크': 2.0,
  'avg': 2.75},
 '나원탁': {'택시운전사': 3.0,
  '남한산성': 4.0,
  '꾼': 3.0,
  '범죄도시': 5.0,
  '아이 캔 스피크': 3.5,
  'avg': 3.7},
 '소이현': {'남한산성': 4.5, '아이 캔 스피크': 1.0, '범죄도시': 4.0, 'avg': 3.1666666666666665}}

In [36]:
def getRecommendation (data, person, k=3, sim_function=sim_pearson):
    
    result = top_match(data, person, k)
    
    score = 0 # 평점 합을 위한 변수
    li = list() # 리턴을 위한 리스트
    score_dic = dict() # 유사도 총합을 위한 dic
    sim_dic = dict() # 평점 총합을 위한 dic

    for sim, name in result: # 튜플이므로 한번에
        print(sim, name)
        if sim < 0 : continue #유사도가 양수인 사람만
        for movie in data[name]: 
            if movie not in data[person]: #name이 평가를 내리지 않은 영화
                score += sim * (data[name][movie] - data[name]['avg']) # 그사람의 영화평점 * 유사도
                score_dic.setdefault(movie, 0) # 기본값 설정
                score_dic[movie] += score # 합계 구함

                # 조건에 맞는 사람의 유사도의 누적합을 구한다
                sim_dic.setdefault(movie, 0) 
                sim_dic[movie] += sim

            score = 0  #영화가 바뀌었으니 초기화한다
    
    for key in score_dic: 
        score_dic[key] = data[person]['avg'] + (score_dic[key] / sim_dic[key]) # 평점 총합/ 유사도 총합
        li.append((score_dic[key],key)) # list((tuple))의 리턴을 위해서.
    li.sort() #오름차순
    li.reverse() #내림차순
    return li

In [37]:
getRecommendation(ratings_expand, '소이현')

0.9661614876137535 홍수환
0.9438405255065262 최홍만
0.9437757473484876 설경구


[(3.91667234143915, '꾼'),
 (2.7989920841776077, '택시운전사'),
 (2.0343626946178484, '킹스맨:골든서클')]

In [38]:
getRecommendation(ratings_expand, '최홍만', k=2, sim_function=sim_cosine)

0.9652129842456227 홍수환
0.9438405255065262 소이현


[(3.75, '택시운전사')]